In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import lda
from sklearn.externals import joblib
import time

In [ ]:
header = ['class_id', 'title', 'description']
train_df = pd.read_csv('../data/ag_news/train.csv', header=None, names=header)

In [ ]:
train_df.head()

In [ ]:
train_df.title = train_df.title.str.lower()
train_df.description = train_df.description.str.lower()

In [ ]:
train_x = train_df.title.values + ' ' + train_df.description.values

In [ ]:
# bow_model = CountVectorizer(stop_words='english')

In [ ]:
# bow = bow_model.fit_transform(train_x)

In [ ]:
# bow_model.get_feature_names()[:10]

In [ ]:
# joblib.dump(bow_model, '../models/bow_model.pkl')
bow_model = joblib.load('../models/bow_model.pkl')

In [ ]:
bow = bow_model.transform(train_x)

In [ ]:
n_topics = [100]
n_iter = 2000

In [ ]:
with open('./topic100iter2000_done_time.txt', 'w') as f:
    for n in n_topics:
        start = time.time()
        lda_model = lda.lda.LDA(n_topics=n, n_iter=n_iter, random_state=0)
        lda_model.fit(bow)
        joblib.dump(lda_model, '../models/lda_model_{}_{}iter.pkl'.format(n, n_iter))
        end = time.time()
        print("topic_N =", str(n), "train time", end - start, file=f)

In [ ]:
lda_model_100 = joblib.load('../models/lda_model_100_2000iter.pkl')

In [ ]:
feature_names = bow_model.get_feature_names()

In [ ]:
for topic in lda_model_100.components_:
    sorted_index = np.argsort(topic)[::-1]
    top_words = np.array([feature_names[i] for i in sorted_index[:20]])
    print(top_words)

In [ ]:
bow = bow_model.transform(train_x)
theta_docs_100 = lda_model_100.transform(bow)

In [ ]:
topic_100_df = pd.DataFrame(theta_docs_100)

topic_100_df.columns = ['topic' + str(i) for i in range(100)]

pd.concat([train_df, topic_100_df], axis=1).to_csv('../data/ag_news/topic100_train.csv', index=False)

In [ ]:
test_df = pd.read_csv('../data/ag_news/test.csv', header=None, names=header)

test_df.title = test_df.title.str.lower()
test_df.description = test_df.description.str.lower()

test_x = test_df.title.values + ' ' + test_df.description.values

In [ ]:
bow = bow_model.transform(test_x)
theta_docs_100 = lda_model_100.transform(bow)

In [ ]:
topic_100_df = pd.DataFrame(theta_docs_100)

topic_100_df.columns = ['topic' + str(i) for i in range(100)]

pd.concat([test_df, topic_100_df], axis=1).to_csv('../data/ag_news/topic100_test.csv', index=False)